In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
from scraper import fetch_website_contents
import gradio as gr

In [2]:
load_dotenv(override=True)
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
openai = OpenAI()
openAIModel = "gpt-4.1-mini"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
ollamaModel = "gpt-oss:latest"

In [3]:
system_message = "You are a helpful assistant"

def message_ollama(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = ollama.chat.completions.create(model=ollamaModel, messages=messages)
    return response.choices[0].message.content

In [4]:
message_ollama("hello ollama")

'Hello! 👋 I’m ChatGPT, here to help you with any questions or projects you’ve got—whether it’s about Ollama, AI in general, coding, or just a casual chat. What’s on your mind today?'

In [5]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [6]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=ollamaModel, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

TypeError: ChatInterface.__init__() got an unexpected keyword argument 'type'

In [ ]:
# Let's start by making a useful function

ticket_prices = {"australia": "$599", "thailand": "$299", "tokyo": "$1400", "india": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"


In [ ]:
get_ticket_price("Australia")

Tool called for city Australia


'The price of a ticket to Australia is $599'

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=ollamaModel, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = ollama.chat.completions.create(model=ollamaModel, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name == "get_ticket_price":
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price_details = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": price_details,
            "tool_call_id": tool_call.id
        }
    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [ ]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=ollamaModel, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = ollama.chat.completions.create(model=ollamaModel, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


Tool called for city India
Tool called for city India
Tool called for city Paris
